In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [2]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

120
500
1000


# GSAGE

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [4]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [6]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

[  0   2   4   8  11  13  15  19  21  26  32  35  37  39  41  44  45  46
  50  53  58  59  60  61  63  65  66  70  71  72  73  74  75  76  77  78
  83  85  86  88  89  92  94  95  96 101 103 104 105 107 108 110 113 115
 116 119 120 121 123 124 125 126 130 131 133 135 137 139 140 141 142 144
 145 146 148 152 153 156 161 168 170 171 173 177 179 181 182]


In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8516, Train: 70.11%, Valid: 62.71% Test: 56.76%
Run: 01, Epoch: 02, Loss: 0.9066, Train: 82.76%, Valid: 71.19% Test: 67.57%
Run: 01, Epoch: 03, Loss: 0.6393, Train: 87.36%, Valid: 72.88% Test: 75.68%
Run: 01, Epoch: 04, Loss: 0.5655, Train: 88.51%, Valid: 74.58% Test: 78.38%
Run: 01, Epoch: 05, Loss: 0.4842, Train: 90.80%, Valid: 74.58% Test: 78.38%
Run: 01, Epoch: 06, Loss: 0.4468, Train: 93.10%, Valid: 71.19% Test: 78.38%
Run: 01, Epoch: 07, Loss: 0.3503, Train: 94.25%, Valid: 72.88% Test: 78.38%
Run: 01, Epoch: 08, Loss: 0.3169, Train: 95.40%, Valid: 72.88% Test: 78.38%
Run: 01, Epoch: 09, Loss: 0.3446, Train: 95.40%, Valid: 74.58% Test: 81.08%
Run: 01, Epoch: 10, Loss: 0.2925, Train: 97.70%, Valid: 74.58% Test: 81.08%
Run: 01, Epoch: 11, Loss: 0.2317, Train: 97.70%, Valid: 74.58% Test: 78.38%
Run: 01, Epoch: 12, Loss: 0.2311, Train: 98.85%, Valid: 74.58% Test: 78.38%
Run: 01, Epoch: 13, Loss: 0.1854, Train: 98.85%, Valid: 76.27% Test: 78.38%
Run: 01, Epo

Run: 02, Epoch: 33, Loss: 0.0795, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 34, Loss: 0.0482, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 35, Loss: 0.0573, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 36, Loss: 0.0688, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 02, Epoch: 37, Loss: 0.0554, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 02, Epoch: 38, Loss: 0.0636, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 02, Epoch: 39, Loss: 0.0545, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 02, Epoch: 40, Loss: 0.0408, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 41, Loss: 0.0458, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 42, Loss: 0.0515, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 43, Loss: 0.0368, Train: 100.00%, Valid: 74.58% Test: 78.38%
Run: 02, Epoch: 44, Loss: 0.0708, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 02, Epoch: 45, Loss: 0.0454, Train: 100.00%, Valid: 76.27% Test: 78.38%

Run: 03, Epoch: 76, Loss: 0.0049, Train: 100.00%, Valid: 74.58% Test: 64.86%
Run: 03, Epoch: 77, Loss: 0.0100, Train: 100.00%, Valid: 74.58% Test: 64.86%
Run: 03, Epoch: 78, Loss: 0.0167, Train: 100.00%, Valid: 74.58% Test: 64.86%
Run: 03, Epoch: 79, Loss: 0.0139, Train: 100.00%, Valid: 74.58% Test: 64.86%
Run: 03, Epoch: 80, Loss: 0.0093, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 81, Loss: 0.0057, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 82, Loss: 0.0054, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 83, Loss: 0.0091, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 84, Loss: 0.0109, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 85, Loss: 0.0106, Train: 100.00%, Valid: 76.27% Test: 64.86%
Run: 03, Epoch: 86, Loss: 0.0092, Train: 100.00%, Valid: 77.97% Test: 64.86%
Run: 03, Epoch: 87, Loss: 0.0042, Train: 100.00%, Valid: 79.66% Test: 64.86%
Run: 03, Epoch: 88, Loss: 0.0119, Train: 100.00%, Valid: 79.66% Test: 64.86%

Run: 05, Epoch: 12, Loss: 0.2393, Train: 97.70%, Valid: 61.02% Test: 70.27%
Run: 05, Epoch: 13, Loss: 0.1916, Train: 98.85%, Valid: 61.02% Test: 70.27%
Run: 05, Epoch: 14, Loss: 0.1727, Train: 98.85%, Valid: 62.71% Test: 67.57%
Run: 05, Epoch: 15, Loss: 0.1623, Train: 98.85%, Valid: 62.71% Test: 70.27%
Run: 05, Epoch: 16, Loss: 0.1411, Train: 98.85%, Valid: 64.41% Test: 70.27%
Run: 05, Epoch: 17, Loss: 0.1428, Train: 98.85%, Valid: 67.80% Test: 70.27%
Run: 05, Epoch: 18, Loss: 0.1461, Train: 98.85%, Valid: 67.80% Test: 62.16%
Run: 05, Epoch: 19, Loss: 0.1573, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 05, Epoch: 20, Loss: 0.0751, Train: 100.00%, Valid: 71.19% Test: 67.57%
Run: 05, Epoch: 21, Loss: 0.0811, Train: 100.00%, Valid: 72.88% Test: 64.86%
Run: 05, Epoch: 22, Loss: 0.0891, Train: 100.00%, Valid: 72.88% Test: 64.86%
Run: 05, Epoch: 23, Loss: 0.0863, Train: 100.00%, Valid: 72.88% Test: 64.86%
Run: 05, Epoch: 24, Loss: 0.0681, Train: 100.00%, Valid: 72.88% Test: 67.57%
Run: 0

Run: 06, Epoch: 58, Loss: 0.0230, Train: 100.00%, Valid: 76.27% Test: 81.08%
Run: 06, Epoch: 59, Loss: 0.0215, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 60, Loss: 0.0173, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 61, Loss: 0.0217, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 62, Loss: 0.0150, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 63, Loss: 0.0286, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 64, Loss: 0.0178, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 65, Loss: 0.0371, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 66, Loss: 0.0296, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 67, Loss: 0.0600, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 68, Loss: 0.0188, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 69, Loss: 0.0438, Train: 100.00%, Valid: 76.27% Test: 78.38%
Run: 06, Epoch: 70, Loss: 0.0423, Train: 100.00%, Valid: 76.27% Test: 78.38%

Run: 07, Epoch: 95, Loss: 0.0148, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run: 07, Epoch: 96, Loss: 0.0042, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run: 07, Epoch: 97, Loss: 0.0144, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run: 07, Epoch: 98, Loss: 0.0104, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run: 07, Epoch: 99, Loss: 0.0378, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run: 07, Epoch: 100, Loss: 0.0080, Train: 100.00%, Valid: 74.58% Test: 75.68%
Run 07:
Highest Train: 100.00
Highest Valid: 77.97
  Final Train: 97.70
   Final Test: 67.57
Run: 08, Epoch: 01, Loss: 1.7273, Train: 62.07%, Valid: 42.37% Test: 40.54%
Run: 08, Epoch: 02, Loss: 0.8206, Train: 74.71%, Valid: 49.15% Test: 48.65%
Run: 08, Epoch: 03, Loss: 0.7147, Train: 83.91%, Valid: 55.93% Test: 48.65%
Run: 08, Epoch: 04, Loss: 0.5940, Train: 87.36%, Valid: 61.02% Test: 54.05%
Run: 08, Epoch: 05, Loss: 0.4821, Train: 87.36%, Valid: 62.71% Test: 59.46%
Run: 08, Epoch: 06, Loss: 0.4101, Train: 89.66%, Valid: 62.71% T

Run: 09, Epoch: 30, Loss: 0.0414, Train: 100.00%, Valid: 67.80% Test: 81.08%
Run: 09, Epoch: 31, Loss: 0.0496, Train: 100.00%, Valid: 66.10% Test: 81.08%
Run: 09, Epoch: 32, Loss: 0.0466, Train: 100.00%, Valid: 66.10% Test: 81.08%
Run: 09, Epoch: 33, Loss: 0.0292, Train: 100.00%, Valid: 66.10% Test: 81.08%
Run: 09, Epoch: 34, Loss: 0.0768, Train: 100.00%, Valid: 64.41% Test: 81.08%
Run: 09, Epoch: 35, Loss: 0.0738, Train: 100.00%, Valid: 64.41% Test: 81.08%
Run: 09, Epoch: 36, Loss: 0.0585, Train: 100.00%, Valid: 64.41% Test: 81.08%
Run: 09, Epoch: 37, Loss: 0.0287, Train: 100.00%, Valid: 61.02% Test: 81.08%
Run: 09, Epoch: 38, Loss: 0.0635, Train: 100.00%, Valid: 61.02% Test: 81.08%
Run: 09, Epoch: 39, Loss: 0.0332, Train: 100.00%, Valid: 61.02% Test: 81.08%
Run: 09, Epoch: 40, Loss: 0.0283, Train: 100.00%, Valid: 61.02% Test: 81.08%
Run: 09, Epoch: 41, Loss: 0.0165, Train: 100.00%, Valid: 61.02% Test: 81.08%
Run: 09, Epoch: 42, Loss: 0.0659, Train: 100.00%, Valid: 62.71% Test: 81.08%

Run: 10, Epoch: 62, Loss: 0.0210, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 63, Loss: 0.0218, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 64, Loss: 0.0441, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 65, Loss: 0.0219, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 66, Loss: 0.0097, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 67, Loss: 0.0143, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 68, Loss: 0.0269, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 69, Loss: 0.0192, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 70, Loss: 0.0067, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 71, Loss: 0.0589, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 72, Loss: 0.0072, Train: 100.00%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 73, Loss: 0.0065, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 74, Loss: 0.0204, Train: 100.00%, Valid: 69.49% Test: 75.68%

# WISE EMBEDDING

In [8]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [9]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [10]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [11]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [13]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [15]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 39.,  20.,  40.,  ...,  40.,  30.,  32.],
        [231.,  25., 155.,  ..., 155.,  64.,  92.],
        [ 25.,  20.,  26.,  ...,  26.,  23.,  23.],
        ...,
        [ 77.,  32.,  73.,  ...,  73.,  49.,  67.],
        [ 61.,  28.,  56.,  ...,  56.,  54.,  51.],
        [ 41.,  23.,  37.,  ...,  37.,  36.,  32.]])


'torch.FloatTensor'

# WISE EMBEDDING

In [16]:
data.x=CC_domain
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [20]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 32, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-6, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = SAGE(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8323, Train: 50.57%, Valid: 55.93% Test: 62.16%
Run: 01, Epoch: 02, Loss: 1.5578, Train: 32.18%, Valid: 28.81% Test: 45.95%
Run: 01, Epoch: 03, Loss: 1.3837, Train: 25.29%, Valid: 28.81% Test: 13.51%
Run: 01, Epoch: 04, Loss: 1.1562, Train: 35.63%, Valid: 30.51% Test: 18.92%
Run: 01, Epoch: 05, Loss: 1.1102, Train: 36.78%, Valid: 32.20% Test: 21.62%
Run: 01, Epoch: 06, Loss: 1.0226, Train: 35.63%, Valid: 28.81% Test: 27.03%
Run: 01, Epoch: 07, Loss: 0.9775, Train: 35.63%, Valid: 22.03% Test: 29.73%
Run: 01, Epoch: 08, Loss: 0.9065, Train: 37.93%, Valid: 27.12% Test: 29.73%
Run: 01, Epoch: 09, Loss: 0.8176, Train: 43.68%, Valid: 38.98% Test: 37.84%
Run: 01, Epoch: 10, Loss: 0.8674, Train: 54.02%, Valid: 42.37% Test: 40.54%
Run: 01, Epoch: 11, Loss: 0.7775, Train: 57.47%, Valid: 55.93% Test: 43.24%
Run: 01, Epoch: 12, Loss: 0.8891, Train: 64.37%, Valid: 61.02% Test: 51.35%
Run: 01, Epoch: 13, Loss: 0.7206, Train: 73.56%, Valid: 67.80% Test: 64.86%
Run: 01, Epo

Run: 02, Epoch: 82, Loss: 0.1493, Train: 94.25%, Valid: 91.53% Test: 86.49%
Run: 02, Epoch: 83, Loss: 0.2097, Train: 96.55%, Valid: 93.22% Test: 86.49%
Run: 02, Epoch: 84, Loss: 0.2039, Train: 97.70%, Valid: 96.61% Test: 89.19%
Run: 02, Epoch: 85, Loss: 0.1672, Train: 98.85%, Valid: 96.61% Test: 94.59%
Run: 02, Epoch: 86, Loss: 0.1789, Train: 97.70%, Valid: 96.61% Test: 94.59%
Run: 02, Epoch: 87, Loss: 0.1417, Train: 98.85%, Valid: 94.92% Test: 94.59%
Run: 02, Epoch: 88, Loss: 0.2361, Train: 96.55%, Valid: 94.92% Test: 89.19%
Run: 02, Epoch: 89, Loss: 0.1597, Train: 96.55%, Valid: 93.22% Test: 86.49%
Run: 02, Epoch: 90, Loss: 0.1618, Train: 95.40%, Valid: 86.44% Test: 86.49%
Run: 02, Epoch: 91, Loss: 0.1594, Train: 90.80%, Valid: 88.14% Test: 81.08%
Run: 02, Epoch: 92, Loss: 0.1236, Train: 88.51%, Valid: 86.44% Test: 81.08%
Run: 02, Epoch: 93, Loss: 0.1469, Train: 85.06%, Valid: 88.14% Test: 81.08%
Run: 02, Epoch: 94, Loss: 0.1897, Train: 87.36%, Valid: 86.44% Test: 81.08%
Run: 02, Epo

Run: 04, Epoch: 90, Loss: 0.1171, Train: 93.10%, Valid: 89.83% Test: 94.59%
Run: 04, Epoch: 91, Loss: 0.0719, Train: 95.40%, Valid: 89.83% Test: 97.30%
Run: 04, Epoch: 92, Loss: 0.1574, Train: 95.40%, Valid: 89.83% Test: 94.59%
Run: 04, Epoch: 93, Loss: 0.0934, Train: 95.40%, Valid: 89.83% Test: 94.59%
Run: 04, Epoch: 94, Loss: 0.1080, Train: 96.55%, Valid: 89.83% Test: 91.89%
Run: 04, Epoch: 95, Loss: 0.1115, Train: 96.55%, Valid: 91.53% Test: 91.89%
Run: 04, Epoch: 96, Loss: 0.1508, Train: 100.00%, Valid: 93.22% Test: 94.59%
Run: 04, Epoch: 97, Loss: 0.1330, Train: 98.85%, Valid: 93.22% Test: 91.89%
Run: 04, Epoch: 98, Loss: 0.0855, Train: 98.85%, Valid: 93.22% Test: 91.89%
Run: 04, Epoch: 99, Loss: 0.0834, Train: 98.85%, Valid: 93.22% Test: 91.89%
Run: 04, Epoch: 100, Loss: 0.1232, Train: 97.70%, Valid: 91.53% Test: 91.89%
Run 04:
Highest Train: 100.00
Highest Valid: 93.22
  Final Train: 100.00
   Final Test: 94.59
Run: 05, Epoch: 01, Loss: 1.8110, Train: 48.28%, Valid: 57.63% Test:

Run: 07, Epoch: 02, Loss: 1.6298, Train: 22.99%, Valid: 16.95% Test: 29.73%
Run: 07, Epoch: 03, Loss: 1.4124, Train: 31.03%, Valid: 25.42% Test: 27.03%
Run: 07, Epoch: 04, Loss: 1.1510, Train: 27.59%, Valid: 23.73% Test: 16.22%
Run: 07, Epoch: 05, Loss: 1.2653, Train: 27.59%, Valid: 25.42% Test: 16.22%
Run: 07, Epoch: 06, Loss: 1.1568, Train: 25.29%, Valid: 25.42% Test: 10.81%
Run: 07, Epoch: 07, Loss: 0.9167, Train: 24.14%, Valid: 23.73% Test: 10.81%
Run: 07, Epoch: 08, Loss: 0.8875, Train: 26.44%, Valid: 22.03% Test: 13.51%
Run: 07, Epoch: 09, Loss: 0.8137, Train: 32.18%, Valid: 25.42% Test: 13.51%
Run: 07, Epoch: 10, Loss: 0.7976, Train: 56.32%, Valid: 49.15% Test: 43.24%
Run: 07, Epoch: 11, Loss: 0.8216, Train: 64.37%, Valid: 66.10% Test: 56.76%
Run: 07, Epoch: 12, Loss: 0.8349, Train: 74.71%, Valid: 69.49% Test: 56.76%
Run: 07, Epoch: 13, Loss: 0.6866, Train: 73.56%, Valid: 72.88% Test: 62.16%
Run: 07, Epoch: 14, Loss: 0.7236, Train: 74.71%, Valid: 74.58% Test: 62.16%
Run: 07, Epo

Run: 09, Epoch: 30, Loss: 0.4454, Train: 81.61%, Valid: 71.19% Test: 83.78%
Run: 09, Epoch: 31, Loss: 0.4429, Train: 80.46%, Valid: 72.88% Test: 83.78%
Run: 09, Epoch: 32, Loss: 0.4442, Train: 82.76%, Valid: 72.88% Test: 83.78%
Run: 09, Epoch: 33, Loss: 0.4128, Train: 80.46%, Valid: 74.58% Test: 83.78%
Run: 09, Epoch: 34, Loss: 0.4176, Train: 79.31%, Valid: 74.58% Test: 83.78%
Run: 09, Epoch: 35, Loss: 0.3388, Train: 79.31%, Valid: 74.58% Test: 83.78%
Run: 09, Epoch: 36, Loss: 0.3010, Train: 81.61%, Valid: 74.58% Test: 83.78%
Run: 09, Epoch: 37, Loss: 0.3246, Train: 85.06%, Valid: 74.58% Test: 83.78%
Run: 09, Epoch: 38, Loss: 0.3635, Train: 88.51%, Valid: 77.97% Test: 86.49%
Run: 09, Epoch: 39, Loss: 0.3054, Train: 91.95%, Valid: 77.97% Test: 83.78%
Run: 09, Epoch: 40, Loss: 0.2907, Train: 94.25%, Valid: 81.36% Test: 83.78%
Run: 09, Epoch: 41, Loss: 0.3363, Train: 94.25%, Valid: 79.66% Test: 86.49%
Run: 09, Epoch: 42, Loss: 0.3433, Train: 93.10%, Valid: 81.36% Test: 89.19%
Run: 09, Epo

# Topological ENCODDING 

In [4]:
dataset = WebKB(root='/tmp/Texas', name='Texas')
data = dataset[0]
print(data)

Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])


In [5]:
print(data.edge_index.numpy())

[[  0   0   1   2   4   4   4   7   7  10  11  11  13  13  15  15  15  15
   15  17  18  20  20  20  20  23  23  25  25  28  28  29  29  29  30  30
   34  34  34  36  36  39  40  41  41  41  41  42  42  44  45  45  46  47
   47  50  50  50  51  53  54  55  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56  56
   56  56  56  56  57  57  57  57  57  57  57  58  58  58  58  58  59  59
   60  60  61  63  64  66  67  70  72  73  73  73  74  74  75  78  78  79
   80  81  82  82  82  82  83  83  83  84  84  84  84  84  84  84  84  85
   86  87  88  89  90  90  92  95  95  95  95  95  99  99  99 100 101 102
  102 102 105 106 107 108 108 108 111 

In [7]:
Number_nodes=len(data.y)
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [8]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [9]:
print(G.number_of_edges())

325


In [10]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [11]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [12]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  60 

2  |  42 

3  |  33 

4  |  19 

5  |  7 

6  |  2 

7  |  4 

8  |  5 

9  |  4 

10  |  2 

11  |  1 

12  |  1 

13  |  1 

20  |  1 

104  |  1 



In [13]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,100]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 182 (100%)

In [14]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [15]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [33]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325], topo=[183, 24])


# TOPO-W-GSAGE

In [16]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [38]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.6149, Train: 48.28%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 02, Loss: 1.5501, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 03, Loss: 1.4962, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 04, Loss: 1.4376, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 05, Loss: 1.4008, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 06, Loss: 1.3555, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 07, Loss: 1.2673, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 08, Loss: 1.2453, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 09, Loss: 1.1866, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 10, Loss: 1.1425, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 11, Loss: 1.1071, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 12, Loss: 1.0309, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 13, Loss: 1.0598, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epo

Run: 01, Epoch: 120, Loss: 0.2435, Train: 100.00%, Valid: 91.53% Test: 91.89%
Run: 01, Epoch: 121, Loss: 0.2742, Train: 100.00%, Valid: 91.53% Test: 89.19%
Run: 01, Epoch: 122, Loss: 0.3262, Train: 100.00%, Valid: 91.53% Test: 89.19%
Run: 01, Epoch: 123, Loss: 0.2966, Train: 100.00%, Valid: 93.22% Test: 91.89%
Run: 01, Epoch: 124, Loss: 0.2764, Train: 100.00%, Valid: 93.22% Test: 94.59%
Run: 01, Epoch: 125, Loss: 0.2114, Train: 98.85%, Valid: 94.92% Test: 94.59%
Run: 01, Epoch: 126, Loss: 0.2392, Train: 100.00%, Valid: 96.61% Test: 94.59%
Run: 01, Epoch: 127, Loss: 0.2815, Train: 100.00%, Valid: 96.61% Test: 91.89%
Run: 01, Epoch: 128, Loss: 0.2748, Train: 100.00%, Valid: 96.61% Test: 94.59%
Run: 01, Epoch: 129, Loss: 0.2534, Train: 100.00%, Valid: 96.61% Test: 94.59%
Run: 01, Epoch: 130, Loss: 0.2218, Train: 97.70%, Valid: 94.92% Test: 94.59%
Run: 01, Epoch: 131, Loss: 0.2516, Train: 96.55%, Valid: 94.92% Test: 97.30%
Run: 01, Epoch: 132, Loss: 0.2053, Train: 96.55%, Valid: 93.22% Tes

Run: 02, Epoch: 49, Loss: 0.5821, Train: 79.31%, Valid: 67.80% Test: 75.68%
Run: 02, Epoch: 50, Loss: 0.6255, Train: 73.56%, Valid: 64.41% Test: 75.68%
Run: 02, Epoch: 51, Loss: 0.6046, Train: 73.56%, Valid: 66.10% Test: 72.97%
Run: 02, Epoch: 52, Loss: 0.5852, Train: 73.56%, Valid: 66.10% Test: 72.97%
Run: 02, Epoch: 53, Loss: 0.5619, Train: 74.71%, Valid: 66.10% Test: 72.97%
Run: 02, Epoch: 54, Loss: 0.5634, Train: 72.41%, Valid: 66.10% Test: 72.97%
Run: 02, Epoch: 55, Loss: 0.5762, Train: 77.01%, Valid: 67.80% Test: 72.97%
Run: 02, Epoch: 56, Loss: 0.5655, Train: 78.16%, Valid: 67.80% Test: 75.68%
Run: 02, Epoch: 57, Loss: 0.5512, Train: 79.31%, Valid: 67.80% Test: 78.38%
Run: 02, Epoch: 58, Loss: 0.5571, Train: 78.16%, Valid: 67.80% Test: 78.38%
Run: 02, Epoch: 59, Loss: 0.5576, Train: 79.31%, Valid: 67.80% Test: 78.38%
Run: 02, Epoch: 60, Loss: 0.5436, Train: 79.31%, Valid: 67.80% Test: 78.38%
Run: 02, Epoch: 61, Loss: 0.5074, Train: 80.46%, Valid: 69.49% Test: 78.38%
Run: 02, Epo

Run: 02, Epoch: 177, Loss: 0.2187, Train: 98.85%, Valid: 96.61% Test: 91.89%
Run: 02, Epoch: 178, Loss: 0.2560, Train: 100.00%, Valid: 96.61% Test: 91.89%
Run: 02, Epoch: 179, Loss: 0.2505, Train: 98.85%, Valid: 96.61% Test: 91.89%
Run: 02, Epoch: 180, Loss: 0.2800, Train: 100.00%, Valid: 96.61% Test: 89.19%
Run: 02, Epoch: 181, Loss: 0.2469, Train: 98.85%, Valid: 94.92% Test: 89.19%
Run: 02, Epoch: 182, Loss: 0.2015, Train: 98.85%, Valid: 89.83% Test: 86.49%
Run: 02, Epoch: 183, Loss: 0.2390, Train: 98.85%, Valid: 91.53% Test: 86.49%
Run: 02, Epoch: 184, Loss: 0.2786, Train: 96.55%, Valid: 89.83% Test: 86.49%
Run: 02, Epoch: 185, Loss: 0.2757, Train: 95.40%, Valid: 91.53% Test: 86.49%
Run: 02, Epoch: 186, Loss: 0.2882, Train: 96.55%, Valid: 91.53% Test: 89.19%
Run: 02, Epoch: 187, Loss: 0.2960, Train: 97.70%, Valid: 98.31% Test: 86.49%
Run: 02, Epoch: 188, Loss: 0.2221, Train: 96.55%, Valid: 98.31% Test: 86.49%
Run: 02, Epoch: 189, Loss: 0.2067, Train: 96.55%, Valid: 98.31% Test: 86.4

Run: 03, Epoch: 108, Loss: 0.2729, Train: 100.00%, Valid: 94.92% Test: 86.49%
Run: 03, Epoch: 109, Loss: 0.2609, Train: 100.00%, Valid: 96.61% Test: 86.49%
Run: 03, Epoch: 110, Loss: 0.2958, Train: 100.00%, Valid: 98.31% Test: 83.78%
Run: 03, Epoch: 111, Loss: 0.3287, Train: 100.00%, Valid: 96.61% Test: 81.08%
Run: 03, Epoch: 112, Loss: 0.2247, Train: 98.85%, Valid: 96.61% Test: 86.49%
Run: 03, Epoch: 113, Loss: 0.2587, Train: 98.85%, Valid: 96.61% Test: 89.19%
Run: 03, Epoch: 114, Loss: 0.2547, Train: 98.85%, Valid: 98.31% Test: 91.89%
Run: 03, Epoch: 115, Loss: 0.2192, Train: 98.85%, Valid: 100.00% Test: 89.19%
Run: 03, Epoch: 116, Loss: 0.2348, Train: 98.85%, Valid: 100.00% Test: 89.19%
Run: 03, Epoch: 117, Loss: 0.2441, Train: 97.70%, Valid: 98.31% Test: 91.89%
Run: 03, Epoch: 118, Loss: 0.3031, Train: 96.55%, Valid: 94.92% Test: 91.89%
Run: 03, Epoch: 119, Loss: 0.2369, Train: 96.55%, Valid: 94.92% Test: 86.49%
Run: 03, Epoch: 120, Loss: 0.2563, Train: 97.70%, Valid: 93.22% Test: 

...

Run: 07, Epoch: 185, Loss: 0.1669, Train: 95.40%, Valid: 86.44% Test: 89.19%
Run: 07, Epoch: 186, Loss: 0.2120, Train: 96.55%, Valid: 88.14% Test: 91.89%
Run: 07, Epoch: 187, Loss: 0.2607, Train: 97.70%, Valid: 93.22% Test: 91.89%
Run: 07, Epoch: 188, Loss: 0.1890, Train: 97.70%, Valid: 93.22% Test: 94.59%
Run: 07, Epoch: 189, Loss: 0.1841, Train: 98.85%, Valid: 91.53% Test: 91.89%
Run: 07, Epoch: 190, Loss: 0.1800, Train: 98.85%, Valid: 89.83% Test: 89.19%
Run: 07, Epoch: 191, Loss: 0.2315, Train: 98.85%, Valid: 89.83% Test: 91.89%
Run: 07, Epoch: 192, Loss: 0.2143, Train: 98.85%, Valid: 91.53% Test: 94.59%
Run: 07, Epoch: 193, Loss: 0.1550, Train: 98.85%, Valid: 91.53% Test: 91.89%
Run: 07, Epoch: 194, Loss: 0.1410, Train: 98.85%, Valid: 93.22% Test: 94.59%
Run: 07, Epoch: 195, Loss: 0.1909, Train: 98.85%, Valid: 93.22% Test: 94.59%
Run: 07, Epoch: 196, Loss: 0.1906, Train: 98.85%, Valid: 91.53% Test: 91.89%
Run: 07, Epoch: 197, Loss: 0.2715, Train: 97.70%, Valid: 94.92% Test: 91.89%

Run: 08, Epoch: 108, Loss: 0.3066, Train: 98.85%, Valid: 94.92% Test: 97.30%
Run: 08, Epoch: 109, Loss: 0.2896, Train: 100.00%, Valid: 93.22% Test: 94.59%
Run: 08, Epoch: 110, Loss: 0.3410, Train: 98.85%, Valid: 89.83% Test: 91.89%
Run: 08, Epoch: 111, Loss: 0.3181, Train: 98.85%, Valid: 89.83% Test: 91.89%
Run: 08, Epoch: 112, Loss: 0.2793, Train: 98.85%, Valid: 89.83% Test: 91.89%
Run: 08, Epoch: 113, Loss: 0.2609, Train: 98.85%, Valid: 91.53% Test: 91.89%
Run: 08, Epoch: 114, Loss: 0.2615, Train: 97.70%, Valid: 94.92% Test: 94.59%
Run: 08, Epoch: 115, Loss: 0.2807, Train: 97.70%, Valid: 91.53% Test: 91.89%
Run: 08, Epoch: 116, Loss: 0.2781, Train: 95.40%, Valid: 88.14% Test: 89.19%
Run: 08, Epoch: 117, Loss: 0.2424, Train: 95.40%, Valid: 88.14% Test: 89.19%
Run: 08, Epoch: 118, Loss: 0.3312, Train: 95.40%, Valid: 88.14% Test: 89.19%
Run: 08, Epoch: 119, Loss: 0.2529, Train: 95.40%, Valid: 88.14% Test: 94.59%
Run: 08, Epoch: 120, Loss: 0.2936, Train: 96.55%, Valid: 88.14% Test: 91.89

Run: 09, Epoch: 35, Loss: 0.7487, Train: 68.97%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 36, Loss: 0.7450, Train: 70.11%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 37, Loss: 0.7296, Train: 70.11%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 38, Loss: 0.6619, Train: 68.97%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 39, Loss: 0.7293, Train: 68.97%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 40, Loss: 0.6481, Train: 66.67%, Valid: 64.41% Test: 78.38%
Run: 09, Epoch: 41, Loss: 0.6686, Train: 67.82%, Valid: 62.71% Test: 78.38%
Run: 09, Epoch: 42, Loss: 0.6317, Train: 65.52%, Valid: 59.32% Test: 72.97%
Run: 09, Epoch: 43, Loss: 0.6465, Train: 63.22%, Valid: 55.93% Test: 72.97%
Run: 09, Epoch: 44, Loss: 0.6595, Train: 60.92%, Valid: 54.24% Test: 72.97%
Run: 09, Epoch: 45, Loss: 0.6744, Train: 60.92%, Valid: 54.24% Test: 72.97%
Run: 09, Epoch: 46, Loss: 0.6132, Train: 60.92%, Valid: 55.93% Test: 72.97%
Run: 09, Epoch: 47, Loss: 0.5875, Train: 64.37%, Valid: 61.02% Test: 75.68%
Run: 09, Epo

Run: 09, Epoch: 161, Loss: 0.2386, Train: 96.55%, Valid: 88.14% Test: 89.19%
Run: 09, Epoch: 162, Loss: 0.3055, Train: 96.55%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 163, Loss: 0.2789, Train: 96.55%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 164, Loss: 0.2566, Train: 96.55%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 165, Loss: 0.2340, Train: 96.55%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 166, Loss: 0.2459, Train: 96.55%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 167, Loss: 0.2763, Train: 97.70%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 168, Loss: 0.2691, Train: 97.70%, Valid: 91.53% Test: 89.19%
Run: 09, Epoch: 169, Loss: 0.2514, Train: 98.85%, Valid: 91.53% Test: 89.19%
Run: 09, Epoch: 170, Loss: 0.2880, Train: 98.85%, Valid: 89.83% Test: 89.19%
Run: 09, Epoch: 171, Loss: 0.2905, Train: 98.85%, Valid: 89.83% Test: 91.89%
Run: 09, Epoch: 172, Loss: 0.2343, Train: 98.85%, Valid: 91.53% Test: 91.89%
Run: 09, Epoch: 173, Loss: 0.2690, Train: 98.85%, Valid: 94.92% Test: 91.89%

Run: 10, Epoch: 87, Loss: 0.4408, Train: 96.55%, Valid: 86.44% Test: 94.59%
Run: 10, Epoch: 88, Loss: 0.3623, Train: 96.55%, Valid: 88.14% Test: 94.59%
Run: 10, Epoch: 89, Loss: 0.3800, Train: 97.70%, Valid: 86.44% Test: 91.89%
Run: 10, Epoch: 90, Loss: 0.4499, Train: 97.70%, Valid: 86.44% Test: 91.89%
Run: 10, Epoch: 91, Loss: 0.3825, Train: 95.40%, Valid: 84.75% Test: 89.19%
Run: 10, Epoch: 92, Loss: 0.3955, Train: 94.25%, Valid: 83.05% Test: 86.49%
Run: 10, Epoch: 93, Loss: 0.3477, Train: 93.10%, Valid: 79.66% Test: 86.49%
Run: 10, Epoch: 94, Loss: 0.4167, Train: 94.25%, Valid: 77.97% Test: 86.49%
Run: 10, Epoch: 95, Loss: 0.3636, Train: 94.25%, Valid: 79.66% Test: 86.49%
Run: 10, Epoch: 96, Loss: 0.4111, Train: 90.80%, Valid: 77.97% Test: 86.49%
Run: 10, Epoch: 97, Loss: 0.3879, Train: 91.95%, Valid: 79.66% Test: 83.78%
Run: 10, Epoch: 98, Loss: 0.3909, Train: 97.70%, Valid: 86.44% Test: 91.89%
Run: 10, Epoch: 99, Loss: 0.3960, Train: 97.70%, Valid: 86.44% Test: 94.59%
Run: 10, Epo

# TOPO-GSAGE

In [17]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325], topo=[183, 24])


In [19]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = SAGE(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8315, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 02, Loss: 1.4186, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 03, Loss: 1.0686, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 04, Loss: 0.9091, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 05, Loss: 0.9059, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 06, Loss: 0.8465, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 07, Loss: 0.7807, Train: 57.47%, Valid: 54.24% Test: 64.86%
Run: 01, Epoch: 08, Loss: 0.7179, Train: 73.56%, Valid: 54.24% Test: 67.57%
Run: 01, Epoch: 09, Loss: 0.7145, Train: 85.06%, Valid: 64.41% Test: 70.27%
Run: 01, Epoch: 10, Loss: 0.6540, Train: 87.36%, Valid: 67.80% Test: 75.68%
Run: 01, Epoch: 11, Loss: 0.5992, Train: 89.66%, Valid: 69.49% Test: 75.68%
Run: 01, Epoch: 12, Loss: 0.5798, Train: 90.80%, Valid: 69.49% Test: 75.68%
Run: 01, Epoch: 13, Loss: 0.5890, Train: 90.80%, Valid: 71.19% Test: 75.68%
Run: 01, Epo

Run: 01, Epoch: 130, Loss: 0.0268, Train: 100.00%, Valid: 77.97% Test: 83.78%
Run: 01, Epoch: 131, Loss: 0.1123, Train: 100.00%, Valid: 77.97% Test: 83.78%
Run: 01, Epoch: 132, Loss: 0.0652, Train: 100.00%, Valid: 77.97% Test: 83.78%
Run: 01, Epoch: 133, Loss: 0.0191, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 134, Loss: 0.0699, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 135, Loss: 0.0847, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 136, Loss: 0.0712, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 137, Loss: 0.0293, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 138, Loss: 0.0853, Train: 100.00%, Valid: 77.97% Test: 81.08%
Run: 01, Epoch: 139, Loss: 0.0344, Train: 100.00%, Valid: 77.97% Test: 78.38%
Run: 01, Epoch: 140, Loss: 0.0449, Train: 100.00%, Valid: 77.97% Test: 78.38%
Run: 01, Epoch: 141, Loss: 0.0651, Train: 100.00%, Valid: 77.97% Test: 78.38%
Run: 01, Epoch: 142, Loss: 0.0598, Train: 100.00%, Valid: 76.27%

Run: 02, Epoch: 55, Loss: 0.0569, Train: 100.00%, Valid: 79.66% Test: 75.68%
Run: 02, Epoch: 56, Loss: 0.0768, Train: 100.00%, Valid: 81.36% Test: 75.68%
Run: 02, Epoch: 57, Loss: 0.0424, Train: 100.00%, Valid: 79.66% Test: 75.68%
Run: 02, Epoch: 58, Loss: 0.0636, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 59, Loss: 0.0582, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 60, Loss: 0.1086, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 61, Loss: 0.0623, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 62, Loss: 0.1022, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 63, Loss: 0.1195, Train: 100.00%, Valid: 74.58% Test: 72.97%
Run: 02, Epoch: 64, Loss: 0.0243, Train: 100.00%, Valid: 74.58% Test: 72.97%
Run: 02, Epoch: 65, Loss: 0.0509, Train: 100.00%, Valid: 76.27% Test: 72.97%
Run: 02, Epoch: 66, Loss: 0.1055, Train: 100.00%, Valid: 74.58% Test: 72.97%
Run: 02, Epoch: 67, Loss: 0.0961, Train: 100.00%, Valid: 74.58% Test: 72.97%

Run: 02, Epoch: 186, Loss: 0.0383, Train: 100.00%, Valid: 67.80% Test: 67.57%
Run: 02, Epoch: 187, Loss: 0.0359, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 02, Epoch: 188, Loss: 0.0155, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 02, Epoch: 189, Loss: 0.0459, Train: 100.00%, Valid: 67.80% Test: 67.57%
Run: 02, Epoch: 190, Loss: 0.0230, Train: 100.00%, Valid: 67.80% Test: 67.57%
Run: 02, Epoch: 191, Loss: 0.0291, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 02, Epoch: 192, Loss: 0.0207, Train: 100.00%, Valid: 69.49% Test: 67.57%
Run: 02, Epoch: 193, Loss: 0.0396, Train: 100.00%, Valid: 69.49% Test: 70.27%
Run: 02, Epoch: 194, Loss: 0.0373, Train: 100.00%, Valid: 69.49% Test: 70.27%
Run: 02, Epoch: 195, Loss: 0.0246, Train: 100.00%, Valid: 69.49% Test: 70.27%
Run: 02, Epoch: 196, Loss: 0.0312, Train: 100.00%, Valid: 69.49% Test: 70.27%
Run: 02, Epoch: 197, Loss: 0.0210, Train: 100.00%, Valid: 69.49% Test: 70.27%
Run: 02, Epoch: 198, Loss: 0.0309, Train: 100.00%, Valid: 69.49%

Run: 03, Epoch: 108, Loss: 0.0371, Train: 100.00%, Valid: 79.66% Test: 56.76%
Run: 03, Epoch: 109, Loss: 0.0246, Train: 100.00%, Valid: 81.36% Test: 54.05%
Run: 03, Epoch: 110, Loss: 0.0209, Train: 100.00%, Valid: 79.66% Test: 54.05%
Run: 03, Epoch: 111, Loss: 0.0275, Train: 100.00%, Valid: 81.36% Test: 54.05%
Run: 03, Epoch: 112, Loss: 0.0403, Train: 100.00%, Valid: 79.66% Test: 51.35%
Run: 03, Epoch: 113, Loss: 0.0223, Train: 100.00%, Valid: 77.97% Test: 51.35%
Run: 03, Epoch: 114, Loss: 0.0168, Train: 100.00%, Valid: 76.27% Test: 51.35%
Run: 03, Epoch: 115, Loss: 0.0179, Train: 100.00%, Valid: 76.27% Test: 48.65%
Run: 03, Epoch: 116, Loss: 0.0459, Train: 100.00%, Valid: 76.27% Test: 45.95%
Run: 03, Epoch: 117, Loss: 0.0345, Train: 100.00%, Valid: 74.58% Test: 45.95%
Run: 03, Epoch: 118, Loss: 0.0248, Train: 100.00%, Valid: 74.58% Test: 45.95%
Run: 03, Epoch: 119, Loss: 0.0056, Train: 100.00%, Valid: 74.58% Test: 48.65%
Run: 03, Epoch: 120, Loss: 0.0596, Train: 100.00%, Valid: 72.88%

...

Run: 08, Epoch: 88, Loss: 0.0148, Train: 100.00%, Valid: 69.49% Test: 78.38%
Run: 08, Epoch: 89, Loss: 0.0582, Train: 100.00%, Valid: 67.80% Test: 78.38%
Run: 08, Epoch: 90, Loss: 0.0486, Train: 100.00%, Valid: 67.80% Test: 78.38%
Run: 08, Epoch: 91, Loss: 0.0737, Train: 100.00%, Valid: 67.80% Test: 78.38%
Run: 08, Epoch: 92, Loss: 0.0407, Train: 100.00%, Valid: 67.80% Test: 78.38%
Run: 08, Epoch: 93, Loss: 0.0606, Train: 100.00%, Valid: 69.49% Test: 75.68%
Run: 08, Epoch: 94, Loss: 0.0771, Train: 100.00%, Valid: 67.80% Test: 75.68%
Run: 08, Epoch: 95, Loss: 0.0488, Train: 100.00%, Valid: 67.80% Test: 75.68%
Run: 08, Epoch: 96, Loss: 0.0517, Train: 100.00%, Valid: 67.80% Test: 75.68%
Run: 08, Epoch: 97, Loss: 0.0192, Train: 100.00%, Valid: 66.10% Test: 81.08%
Run: 08, Epoch: 98, Loss: 0.0293, Train: 100.00%, Valid: 66.10% Test: 78.38%
Run: 08, Epoch: 99, Loss: 0.0801, Train: 100.00%, Valid: 66.10% Test: 81.08%
Run: 08, Epoch: 100, Loss: 0.0354, Train: 100.00%, Valid: 66.10% Test: 78.38

Run: 09, Epoch: 08, Loss: 0.7558, Train: 65.52%, Valid: 50.85% Test: 64.86%
Run: 09, Epoch: 09, Loss: 0.6775, Train: 72.41%, Valid: 55.93% Test: 70.27%
Run: 09, Epoch: 10, Loss: 0.6538, Train: 73.56%, Valid: 61.02% Test: 70.27%
Run: 09, Epoch: 11, Loss: 0.5746, Train: 77.01%, Valid: 62.71% Test: 70.27%
Run: 09, Epoch: 12, Loss: 0.5221, Train: 89.66%, Valid: 62.71% Test: 72.97%
Run: 09, Epoch: 13, Loss: 0.5223, Train: 95.40%, Valid: 61.02% Test: 78.38%
Run: 09, Epoch: 14, Loss: 0.4864, Train: 97.70%, Valid: 59.32% Test: 78.38%
Run: 09, Epoch: 15, Loss: 0.4346, Train: 97.70%, Valid: 55.93% Test: 78.38%
Run: 09, Epoch: 16, Loss: 0.4771, Train: 97.70%, Valid: 57.63% Test: 78.38%
Run: 09, Epoch: 17, Loss: 0.4535, Train: 97.70%, Valid: 59.32% Test: 75.68%
Run: 09, Epoch: 18, Loss: 0.3878, Train: 97.70%, Valid: 57.63% Test: 75.68%
Run: 09, Epoch: 19, Loss: 0.3460, Train: 98.85%, Valid: 59.32% Test: 78.38%
Run: 09, Epoch: 20, Loss: 0.3987, Train: 98.85%, Valid: 59.32% Test: 81.08%
Run: 09, Epo

Run: 09, Epoch: 136, Loss: 0.0543, Train: 100.00%, Valid: 55.93% Test: 67.57%
Run: 09, Epoch: 137, Loss: 0.0401, Train: 100.00%, Valid: 55.93% Test: 67.57%
Run: 09, Epoch: 138, Loss: 0.0968, Train: 100.00%, Valid: 55.93% Test: 67.57%
Run: 09, Epoch: 139, Loss: 0.0335, Train: 100.00%, Valid: 55.93% Test: 67.57%
Run: 09, Epoch: 140, Loss: 0.0568, Train: 100.00%, Valid: 55.93% Test: 70.27%
Run: 09, Epoch: 141, Loss: 0.0882, Train: 100.00%, Valid: 55.93% Test: 70.27%
Run: 09, Epoch: 142, Loss: 0.0255, Train: 100.00%, Valid: 57.63% Test: 70.27%
Run: 09, Epoch: 143, Loss: 0.0471, Train: 100.00%, Valid: 59.32% Test: 70.27%
Run: 09, Epoch: 144, Loss: 0.0429, Train: 100.00%, Valid: 61.02% Test: 72.97%
Run: 09, Epoch: 145, Loss: 0.0085, Train: 100.00%, Valid: 61.02% Test: 75.68%
Run: 09, Epoch: 146, Loss: 0.0295, Train: 100.00%, Valid: 59.32% Test: 78.38%
Run: 09, Epoch: 147, Loss: 0.0244, Train: 100.00%, Valid: 59.32% Test: 78.38%
Run: 09, Epoch: 148, Loss: 0.0149, Train: 100.00%, Valid: 61.02%

Run: 10, Epoch: 58, Loss: 0.0849, Train: 98.85%, Valid: 67.80% Test: 72.97%
Run: 10, Epoch: 59, Loss: 0.0554, Train: 98.85%, Valid: 67.80% Test: 72.97%
Run: 10, Epoch: 60, Loss: 0.0629, Train: 98.85%, Valid: 67.80% Test: 72.97%
Run: 10, Epoch: 61, Loss: 0.0974, Train: 98.85%, Valid: 69.49% Test: 72.97%
Run: 10, Epoch: 62, Loss: 0.0886, Train: 98.85%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 63, Loss: 0.1084, Train: 98.85%, Valid: 69.49% Test: 75.68%
Run: 10, Epoch: 64, Loss: 0.0609, Train: 98.85%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 65, Loss: 0.0607, Train: 98.85%, Valid: 71.19% Test: 75.68%
Run: 10, Epoch: 66, Loss: 0.0541, Train: 98.85%, Valid: 71.19% Test: 72.97%
Run: 10, Epoch: 67, Loss: 0.0773, Train: 98.85%, Valid: 71.19% Test: 70.27%
Run: 10, Epoch: 68, Loss: 0.0759, Train: 98.85%, Valid: 69.49% Test: 70.27%
Run: 10, Epoch: 69, Loss: 0.1411, Train: 98.85%, Valid: 71.19% Test: 72.97%
Run: 10, Epoch: 70, Loss: 0.0821, Train: 98.85%, Valid: 71.19% Test: 72.97%
Run: 10, Epo

Run: 10, Epoch: 183, Loss: 0.0103, Train: 100.00%, Valid: 71.19% Test: 64.86%
Run: 10, Epoch: 184, Loss: 0.0466, Train: 100.00%, Valid: 71.19% Test: 64.86%
Run: 10, Epoch: 185, Loss: 0.0301, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 10, Epoch: 186, Loss: 0.0190, Train: 100.00%, Valid: 69.49% Test: 64.86%
Run: 10, Epoch: 187, Loss: 0.0133, Train: 100.00%, Valid: 67.80% Test: 64.86%
Run: 10, Epoch: 188, Loss: 0.0478, Train: 100.00%, Valid: 67.80% Test: 62.16%
Run: 10, Epoch: 189, Loss: 0.0350, Train: 100.00%, Valid: 67.80% Test: 62.16%
Run: 10, Epoch: 190, Loss: 0.0746, Train: 100.00%, Valid: 67.80% Test: 64.86%
Run: 10, Epoch: 191, Loss: 0.0185, Train: 100.00%, Valid: 66.10% Test: 64.86%
Run: 10, Epoch: 192, Loss: 0.0202, Train: 100.00%, Valid: 64.41% Test: 64.86%
Run: 10, Epoch: 193, Loss: 0.0144, Train: 100.00%, Valid: 64.41% Test: 64.86%
Run: 10, Epoch: 194, Loss: 0.0087, Train: 100.00%, Valid: 64.41% Test: 64.86%
Run: 10, Epoch: 195, Loss: 0.0657, Train: 100.00%, Valid: 64.41%